In [1]:
import os
import sys

sys.path.append(os.path.abspath("../scripts"))
from data_loader import DataLoader

data_loader = DataLoader()
X_train, y_train = data_loader.training_data
X_val, y_val = data_loader.validation_data
X_test, y_test = data_loader.test_data

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the decision tree model
decision_tree = DecisionTreeClassifier()

# Train the model on the preprocessed training data
decision_tree.fit(X_train, y_train)

# Make predictions on the validation set
y_val_pred = decision_tree.predict(X_val)

# Evaluate the model's performance
accuracy = accuracy_score(y_val, y_val_pred)
report = classification_report(y_val, y_val_pred)

print(f"Validation Accuracy: {accuracy}")
print("Classification Report:\n", report)

Validation Accuracy: 0.788434247871334
Classification Report:
               precision    recall  f1-score   support

         0.0       0.88      0.87      0.87     21371
         1.0       0.33      0.34      0.33      3997

    accuracy                           0.79     25368
   macro avg       0.60      0.60      0.60     25368
weighted avg       0.79      0.79      0.79     25368



In [ ]:
#TODO first splits

import matplotlib.pyplot as plt
from sklearn import tree

#Visualize the decision tree model
plt.figure(figsize=(20,10))
tree.plot_tree(decision_tree,
               feature_names=X_train.columns, 
               class_names=["No Diabetes", "Diabetes"])
plt.show()

KeyboardInterrupt: 

In [ ]:
#TODO look at PCA and over- and undersampling

from sklearn.ensemble import RandomForestClassifier

# Initialize the random forest ensemble model
#TODO Adjust max_depth to? random_state?
random_forest = RandomForestClassifier(max_depth=2, random_state=0)

# Train the model on the preprocessed training data
random_forest.fit(X_train, y_train)

# Make predictions on the validation set
y_val_pred = random_forest.predict(X_val)

# Evaluate the model's performance
accuracy = accuracy_score(y_val, y_val_pred)
report = classification_report(y_val, y_val_pred)

print(f"Validation Accuracy: {accuracy}")
print("Classification Report:\n", report)

Validation Accuracy: 0.842439293598234
Classification Report:
               precision    recall  f1-score   support

         0.0       0.84      1.00      0.91     21371
         1.0       0.00      0.00      0.00      3997

    accuracy                           0.84     25368
   macro avg       0.42      0.50      0.46     25368
weighted avg       0.71      0.84      0.77     25368



/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [7]:
import matplotlib.pyplot as plt
from sklearn import tree

#Visualize the random forest model
plt.figure(figsize=(20,10))
tree.plot_tree(random_forest,
               feature_names=X_train.columns, 
               class_names=["No Diabetes", "Diabetes"])
plt.show()

InvalidParameterError: The 'decision_tree' parameter of plot_tree must be an instance of 'sklearn.tree._classes.DecisionTreeClassifier' or an instance of 'sklearn.tree._classes.DecisionTreeRegressor'. Got RandomForestClassifier(max_depth=2, random_state=0) instead.

<Figure size 2000x1000 with 0 Axes>

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

#Initialize baseline models 
#TODO How many neighbours n= ?
#TODO Do I need probability=True

classifiers = {
    "decision tree": DecisionTreeClassifier(),
    "random forest": RandomForestClassifier(max_depth=2, random_state=0)
    #"logisitc regression": LogisticRegression(),
    #"naive bayes":GaussianNB(), 
    #"support vector machines": SVC(probability=True), # SVM with probabilities for soft voting??
    #"k-NN with n = 5": KNeighborsClassifier(n_neighbors=1)
}

# Implement function to display accuracy as performance metric for different classifiers
#TODO habe y_val genommen nicht y_test ?
def evaluate_classifier(e_name, e, X_train, y_train, X_val, y_val):
    y_pred = e.fit(X_train, y_train).predict(X_val)
    acc = accuracy_score(y_val, y_pred)
    print(f'{e_name}: ACC={acc:.2f}')

# Evaluate the classifiers' accuracies
for e_name, e in classifiers.items():
    evaluate_classifier(e_name, e, X_train, y_train, X_val, y_val)

decision tree: ACC=0.79
random forest: ACC=0.84


In [9]:
from sklearn.ensemble import BaggingClassifier

#Use bagging ensemble method as classifier
bagging_classifiers = {f'{e_name} (B)': BaggingClassifier(e) for e_name, e in classifiers.items()}
classifiers.update(bagging_classifiers)

# Evaluate the classifiers' accuracies
for e_name, e in classifiers.items():
    evaluate_classifier(e_name, e, X_train, y_train, X_val, y_val) #TODO changed _test to _val, ok ?

decision tree: ACC=0.79
random forest: ACC=0.84
decision tree (B): ACC=0.83
random forest (B): ACC=0.84


In [10]:
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

#Use boosting ensemble methods as classifiers
classifiers['AdaBoost'] = AdaBoostClassifier()
classifiers['XGBoost'] = XGBClassifier(use_label_encoder=False)

# Evaluate the classifiers' accuracies
for e_name, e in classifiers.items():
    evaluate_classifier(e_name, e, X_train, y_train, X_val, y_val) #TODO changed _test to _val, ok ?

decision tree: ACC=0.79
random forest: ACC=0.84
decision tree (B): ACC=0.83
random forest (B): ACC=0.84


/opt/anaconda3/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost: ACC=0.85


/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:17:42] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost: ACC=0.85


In [ ]:
#n_estimator = ?

In [ ]:
#Use voting ensemble method as classifier
#TODO random_states = ? und weights = ?
voting = VotingClassifier(
    ("lr", LogisticRegression(),
     "nb",GaussianNB(), 
     "svm", SVC(probability=True), # SVM with probabilities for soft voting
     "knn_5" KNeighborsClassifier(n_neighbors=5),                            
     "dt", DecisionTreeClassifier())  
    voting='soft', weights=[1, 2, 1]
)

#TODO Complete voting
#TODO Evaluate the classifiers' accuracies


In [ ]:
#TODO Stacking